In [28]:
import lightgbm as lgb
from sklearn import metrics
import category_encoders as ce

from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import xgboost as xgb

from sklearn.impute import SimpleImputer

df = pd.read_csv('../data/train.csv',dtype={'tipodepropiedad':'category','ciudad':'category','provincia':'category'})
df['fecha'] = pd.to_datetime(df['fecha'])
df['anio'] = df['fecha'].dt.year
df["mes"] = df['fecha'].dt.month
df["dia"] = df['fecha'].dt.day

df = df.drop(columns=["lat","lng"])

In [29]:
import re
def contiene_frase(texto, frase):
    if re.search(frase, texto,re.IGNORECASE):
        return True
    return False

contiene_frase=np.vectorize(contiene_frase)

In [30]:
def cantidad_de_public(data,colum,palabra):
    return data[contiene_frase(data[colum],palabra)].dia.count()

def serie_que_contiene_palabra(data,colum,palabra):
        return contiene_frase(data[colum],palabra)

# Completo los datos faltantes

In [31]:
imp = SimpleImputer(strategy="constant")
df["provincia"] = imp.fit_transform(df[["provincia"]])
df["ciudad"] = imp.fit_transform(df[["ciudad"]])

In [32]:
df['ciudad-provincia'] = df[['ciudad', 'provincia']].apply(lambda x: ', '.join(x), axis=1)

# Busco en las descripciones

In [33]:
busqueda = "descripcion"
df1 = df.copy()
df1[busqueda] = df1[busqueda].fillna(value="-")
df1.shape

(240000, 25)

In [34]:
palabra = "centro"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

25889

In [35]:
palabra = "patio"
palabra2 = "jardin"
palabra3 = "jardín"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)|serie_que_contiene_palabra(df1,busqueda,palabra3)
df1[palabra].value_counts()[1]

100824

In [36]:
palabra = "hermosa"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

21198

In [37]:
palabra = "grande"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

11163

In [38]:
palabra = "universidad"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

4221

In [39]:
palabra = "ilum"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)


20875

In [40]:
palabra = "seguridad"
palabra2 = "vigilancia"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)
df1[palabra].value_counts()[1]

52199

In [41]:
palabra = "tienda"
palabra2 = "comercial"
palabra3 = "local"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)|serie_que_contiene_palabra(df1,busqueda,palabra3)
df1[palabra].value_counts()[1]

39496

In [42]:
palabra = "comedor"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

127434

In [43]:
palabra = "lujo"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

13816

In [44]:
palabra = "servicio"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

87104

In [45]:
palabra = "remodelada"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)
cantidad_de_public(df1,busqueda,palabra)

6379

In [46]:
palabra = "autos"
palabra2 = "auto"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)
df1[palabra].value_counts()[1]

78290

# Busco en las direcciones

In [47]:
busqueda = "direccion"

df1[busqueda] = df1[busqueda].fillna(value="-")

In [52]:
palabra = "avenida"
palabra2 = "av"

df1[palabra] = serie_que_contiene_palabra(df1,busqueda,palabra)|serie_que_contiene_palabra(df1,busqueda,palabra2)
df1[palabra].value_counts()[1]

20304

# Creo el CSV

In [50]:
df1.shape

(240000, 39)

In [49]:
df1.to_csv("../data/features.csv", index=False)

In [51]:
df1

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,universidad,ilum,seguridad,tienda,comedor,lujo,servicio,remodelada,autos,avenida
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,False,False,False,False,True,False,False,False,False,True
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,False,False,False,False,True,False,True,False,True,True
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,False,False,True,True,False,False,True,False,True,False
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,False,False,True,True,False,False,False,False,False,False
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239995,119879,bonita casas de 2 recamaras a 10 minutos del c...,vendo casa en bosques de ica residencial a 10 ...,Casa,BOSQUES,Zinacantepec,Edo. de México,0.0,2.0,2.0,...,False,False,True,False,True,False,False,False,True,False
239996,259178,casa en condominio a 10 min. del centro de toluca,"casa con un jardin amplio, un cuarto de servic...",Casa,Filiberto Navas 325,Toluca,Edo. de México,0.0,3.0,3.0,...,False,False,False,True,False,False,True,False,True,True
239997,131932,nicolas san juan,"departamento con excelente ubicación, muy cerc...",Apartamento,Nicolas San Juan,Benito Juárez,Distrito Federal,20.0,2.0,1.0,...,True,False,True,False,True,False,True,False,True,False
239998,146867,casa sola. javier rojo gomez.,"casa sola, dividida en cuatro departamentos de...",Casa,Javier Rojo Gomez 120,Iztapalapa,Distrito Federal,20.0,4.0,0.0,...,False,False,False,False,False,False,False,False,False,True
